In [1]:
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.mask
import rasterio.warp             # Reproject raster samples
import rasterio.merge
from rasterio.transform import rowcol
from rasterio import features
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
from osgeo import gdal,ogr,osr
import pandas as pd
import shapely
from shapely.geometry import box, Point
import json

import numpy as np               # numerical array manipulation
import time
import os
from PIL import Image
import PIL.ImageDraw
from core.visualize import display_images

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from multiprocessing import Pool

In [2]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/Preprocessing.py)
# Please provide required info in the file before continuing with this notebook. 
# hbh: in this scene,a new config named Preprocessing_within is created to distinguish from the original
from config import Preprocessing   
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import Preprocessing
config = Preprocessing.Configuration()

In [3]:
output_dir=r'D:\lakemapping\2_dataset\sample747'#sample747
raw_img_dir=r'D:\lakemapping\sample750_pad\tif'

area_prefix='area_'
pad_area_prefix='pad_area_'
raw_image_prefix='sample_'
image_prefix='image_'
pad_image_prefix='pad_image_'
area_file_type='.shp'
image_file_type='.tif'
type_num=6

## 提取shapefile
将shapefile按类别存储为单个文件，以便后续裁剪

In [30]:
for i in range(0,type_num):
    writePath=os.path.join(output_dir,r'output{}'.format(i))
    if not os.path.exists(writePath):
        os.makedirs(writePath)
    writePath2=os.path.join(output_dir,r'shp\output{}'.format(ty))
    if not os.path.exists(writePath2):
        os.makedirs(writePath2)

In [15]:
trainingArea = gps.read_file(os.path.join(config.training_base_dir, config.training_area_fn))
for i in tqdm(trainingArea.index):
    fn=trainingArea.loc[i]['file_name']
    ty=trainingArea.loc[i]['type']
    shape=trainingArea[i:i+1]
    shape.to_file(os.path.join(writePath,area_prefix+fn+area_file_type))

  0%|          | 0/747 [00:00<?, ?it/s]

## 裁剪sampe影像，获得image影像

In [34]:
for i in range(0,type_num):
    all_files = []
    for root, dirs, files in os.walk(os.path.join(output_dir,'shp\output{}'.format(i))):
        for file in files:
            if file.endswith(area_file_type)and file.startswith(area_prefix):
                 all_files.append((os.path.join(root, file), file))
                    
    for fullPath, filename in all_files:
        print(filename)
        raster_fn=os.path.join(raw_img_dir,filename.replace(area_prefix,raw_image_prefix).replace(area_file_type,image_file_type))
        print(raster_fn)
        input_raster = gdal.Open(raster_fn)
        in_shp_path=fullPath
        out_raster_path=fullPath.replace(area_prefix,image_prefix).replace(area_file_type,image_file_type).replace(r'\shp','')
        print(out_raster_path)
            # 利用gdal.Warp进行裁剪
            # https://gdal.org/api/python/osgeo.gdal.html#osgeo.gdal.Warp
        result = gdal.Warp(
                out_raster_path,
                input_raster,
                format = 'GTiff',
                cutlineDSName = in_shp_path, # 用于裁剪的矢量
                cropToCutline = True, # 是否使用cutlineDSName的extent作为输出的界线
                dstNodata = -999 # 输出数据的nodata值
                )
        result.FlushCache()
        del result

area_10031.shp
D:\lakemapping\sample750_pad\tif\sample_10031.tif
D:\lakemapping\2_dataset\sample747\output0\image_10031.tif
area_10150.shp
D:\lakemapping\sample750_pad\tif\sample_10150.tif
D:\lakemapping\2_dataset\sample747\output0\image_10150.tif
area_10174.shp
D:\lakemapping\sample750_pad\tif\sample_10174.tif
D:\lakemapping\2_dataset\sample747\output0\image_10174.tif
area_10332.shp
D:\lakemapping\sample750_pad\tif\sample_10332.tif
D:\lakemapping\2_dataset\sample747\output0\image_10332.tif
area_10358.shp
D:\lakemapping\sample750_pad\tif\sample_10358.tif
D:\lakemapping\2_dataset\sample747\output0\image_10358.tif
area_10383.shp
D:\lakemapping\sample750_pad\tif\sample_10383.tif
D:\lakemapping\2_dataset\sample747\output0\image_10383.tif
area_10408.shp
D:\lakemapping\sample750_pad\tif\sample_10408.tif
D:\lakemapping\2_dataset\sample747\output0\image_10408.tif
area_10689.shp
D:\lakemapping\sample750_pad\tif\sample_10689.tif
D:\lakemapping\2_dataset\sample747\output0\image_10689.tif
area_107

area_14206.shp
D:\lakemapping\sample750_pad\tif\sample_14206.tif
D:\lakemapping\2_dataset\sample747\output0\image_14206.tif
area_14422.shp
D:\lakemapping\sample750_pad\tif\sample_14422.tif
D:\lakemapping\2_dataset\sample747\output0\image_14422.tif
area_14449.shp
D:\lakemapping\sample750_pad\tif\sample_14449.tif
D:\lakemapping\2_dataset\sample747\output0\image_14449.tif
area_14494.shp
D:\lakemapping\sample750_pad\tif\sample_14494.tif
D:\lakemapping\2_dataset\sample747\output0\image_14494.tif
area_14689.shp
D:\lakemapping\sample750_pad\tif\sample_14689.tif
D:\lakemapping\2_dataset\sample747\output0\image_14689.tif
area_147.shp
D:\lakemapping\sample750_pad\tif\sample_147.tif
D:\lakemapping\2_dataset\sample747\output0\image_147.tif
area_14735.shp
D:\lakemapping\sample750_pad\tif\sample_14735.tif
D:\lakemapping\2_dataset\sample747\output0\image_14735.tif
area_14753.shp
D:\lakemapping\sample750_pad\tif\sample_14753.tif
D:\lakemapping\2_dataset\sample747\output0\image_14753.tif
area_14781.shp

area_2051.shp
D:\lakemapping\sample750_pad\tif\sample_2051.tif
D:\lakemapping\2_dataset\sample747\output0\image_2051.tif
area_2320.shp
D:\lakemapping\sample750_pad\tif\sample_2320.tif
D:\lakemapping\2_dataset\sample747\output0\image_2320.tif
area_2345.shp
D:\lakemapping\sample750_pad\tif\sample_2345.tif
D:\lakemapping\2_dataset\sample747\output0\image_2345.tif
area_2433.shp
D:\lakemapping\sample750_pad\tif\sample_2433.tif
D:\lakemapping\2_dataset\sample747\output0\image_2433.tif
area_2589.shp
D:\lakemapping\sample750_pad\tif\sample_2589.tif
D:\lakemapping\2_dataset\sample747\output0\image_2589.tif
area_2720.shp
D:\lakemapping\sample750_pad\tif\sample_2720.tif
D:\lakemapping\2_dataset\sample747\output0\image_2720.tif
area_2893.shp
D:\lakemapping\sample750_pad\tif\sample_2893.tif
D:\lakemapping\2_dataset\sample747\output0\image_2893.tif
area_2923.shp
D:\lakemapping\sample750_pad\tif\sample_2923.tif
D:\lakemapping\2_dataset\sample747\output0\image_2923.tif
area_3015.shp
D:\lakemapping\sam

area_7648.shp
D:\lakemapping\sample750_pad\tif\sample_7648.tif
D:\lakemapping\2_dataset\sample747\output0\image_7648.tif
area_7724.shp
D:\lakemapping\sample750_pad\tif\sample_7724.tif
D:\lakemapping\2_dataset\sample747\output0\image_7724.tif
area_7733.shp
D:\lakemapping\sample750_pad\tif\sample_7733.tif
D:\lakemapping\2_dataset\sample747\output0\image_7733.tif
area_7752.shp
D:\lakemapping\sample750_pad\tif\sample_7752.tif
D:\lakemapping\2_dataset\sample747\output0\image_7752.tif
area_7776.shp
D:\lakemapping\sample750_pad\tif\sample_7776.tif
D:\lakemapping\2_dataset\sample747\output0\image_7776.tif
area_7796.shp
D:\lakemapping\sample750_pad\tif\sample_7796.tif
D:\lakemapping\2_dataset\sample747\output0\image_7796.tif
area_7815.shp
D:\lakemapping\sample750_pad\tif\sample_7815.tif
D:\lakemapping\2_dataset\sample747\output0\image_7815.tif
area_7889.shp
D:\lakemapping\sample750_pad\tif\sample_7889.tif
D:\lakemapping\2_dataset\sample747\output0\image_7889.tif
area_7993.shp
D:\lakemapping\sam

area_11562_1.shp
D:\lakemapping\sample750_pad\tif\sample_11562_1.tif
D:\lakemapping\2_dataset\sample747\output1\image_11562_1.tif
area_11562_2.shp
D:\lakemapping\sample750_pad\tif\sample_11562_2.tif
D:\lakemapping\2_dataset\sample747\output1\image_11562_2.tif
area_11581.shp
D:\lakemapping\sample750_pad\tif\sample_11581.tif
D:\lakemapping\2_dataset\sample747\output1\image_11581.tif
area_11624.shp
D:\lakemapping\sample750_pad\tif\sample_11624.tif
D:\lakemapping\2_dataset\sample747\output1\image_11624.tif
area_11632.shp
D:\lakemapping\sample750_pad\tif\sample_11632.tif
D:\lakemapping\2_dataset\sample747\output1\image_11632.tif
area_11676.shp
D:\lakemapping\sample750_pad\tif\sample_11676.tif
D:\lakemapping\2_dataset\sample747\output1\image_11676.tif
area_11765.shp
D:\lakemapping\sample750_pad\tif\sample_11765.tif
D:\lakemapping\2_dataset\sample747\output1\image_11765.tif
area_11814.shp
D:\lakemapping\sample750_pad\tif\sample_11814.tif
D:\lakemapping\2_dataset\sample747\output1\image_11814.

...

area_2827.shp
D:\lakemapping\sample750_pad\tif\sample_2827.tif
D:\lakemapping\2_dataset\sample747\output1\image_2827.tif
area_2896.shp
D:\lakemapping\sample750_pad\tif\sample_2896.tif
D:\lakemapping\2_dataset\sample747\output1\image_2896.tif
area_2913_1.shp
D:\lakemapping\sample750_pad\tif\sample_2913_1.tif
D:\lakemapping\2_dataset\sample747\output1\image_2913_1.tif
area_2913_2.shp
D:\lakemapping\sample750_pad\tif\sample_2913_2.tif
D:\lakemapping\2_dataset\sample747\output1\image_2913_2.tif
area_2922.shp
D:\lakemapping\sample750_pad\tif\sample_2922.tif
D:\lakemapping\2_dataset\sample747\output1\image_2922.tif
area_306.shp
D:\lakemapping\sample750_pad\tif\sample_306.tif
D:\lakemapping\2_dataset\sample747\output1\image_306.tif
area_3090.shp
D:\lakemapping\sample750_pad\tif\sample_3090.tif
D:\lakemapping\2_dataset\sample747\output1\image_3090.tif
area_3458.shp
D:\lakemapping\sample750_pad\tif\sample_3458.tif
D:\lakemapping\2_dataset\sample747\output1\image_3458.tif
area_3531.shp
D:\lakema

area_9618.shp
D:\lakemapping\sample750_pad\tif\sample_9618.tif
D:\lakemapping\2_dataset\sample747\output1\image_9618.tif
area_9816.shp
D:\lakemapping\sample750_pad\tif\sample_9816.tif
D:\lakemapping\2_dataset\sample747\output1\image_9816.tif
area_9847.shp
D:\lakemapping\sample750_pad\tif\sample_9847.tif
D:\lakemapping\2_dataset\sample747\output1\image_9847.tif
area_9875.shp
D:\lakemapping\sample750_pad\tif\sample_9875.tif
D:\lakemapping\2_dataset\sample747\output1\image_9875.tif
area_995.shp
D:\lakemapping\sample750_pad\tif\sample_995.tif
D:\lakemapping\2_dataset\sample747\output1\image_995.tif
area_10171.shp
D:\lakemapping\sample750_pad\tif\sample_10171.tif
D:\lakemapping\2_dataset\sample747\output2\image_10171.tif
area_10198.shp
D:\lakemapping\sample750_pad\tif\sample_10198.tif
D:\lakemapping\2_dataset\sample747\output2\image_10198.tif
area_10203.shp
D:\lakemapping\sample750_pad\tif\sample_10203.tif
D:\lakemapping\2_dataset\sample747\output2\image_10203.tif
area_10228.shp
D:\lakemapp

D:\lakemapping\2_dataset\sample747\output2\image_4631.tif
area_4850.shp
D:\lakemapping\sample750_pad\tif\sample_4850.tif
D:\lakemapping\2_dataset\sample747\output2\image_4850.tif
area_4851.shp
D:\lakemapping\sample750_pad\tif\sample_4851.tif
D:\lakemapping\2_dataset\sample747\output2\image_4851.tif
area_4886.shp
D:\lakemapping\sample750_pad\tif\sample_4886.tif
D:\lakemapping\2_dataset\sample747\output2\image_4886.tif
area_4892.shp
D:\lakemapping\sample750_pad\tif\sample_4892.tif
D:\lakemapping\2_dataset\sample747\output2\image_4892.tif
area_4917.shp
D:\lakemapping\sample750_pad\tif\sample_4917.tif
D:\lakemapping\2_dataset\sample747\output2\image_4917.tif
area_4949.shp
D:\lakemapping\sample750_pad\tif\sample_4949.tif
D:\lakemapping\2_dataset\sample747\output2\image_4949.tif
area_4976.shp
D:\lakemapping\sample750_pad\tif\sample_4976.tif
D:\lakemapping\2_dataset\sample747\output2\image_4976.tif
area_5039.shp
D:\lakemapping\sample750_pad\tif\sample_5039.tif
D:\lakemapping\2_dataset\sample7

area_12746.shp
D:\lakemapping\sample750_pad\tif\sample_12746.tif
D:\lakemapping\2_dataset\sample747\output3\image_12746.tif
area_12770.shp
D:\lakemapping\sample750_pad\tif\sample_12770.tif
D:\lakemapping\2_dataset\sample747\output3\image_12770.tif
area_12793.shp
D:\lakemapping\sample750_pad\tif\sample_12793.tif
D:\lakemapping\2_dataset\sample747\output3\image_12793.tif
area_12810.shp
D:\lakemapping\sample750_pad\tif\sample_12810.tif
D:\lakemapping\2_dataset\sample747\output3\image_12810.tif
area_12847.shp
D:\lakemapping\sample750_pad\tif\sample_12847.tif
D:\lakemapping\2_dataset\sample747\output3\image_12847.tif
area_12889.shp
D:\lakemapping\sample750_pad\tif\sample_12889.tif
D:\lakemapping\2_dataset\sample747\output3\image_12889.tif
area_12903.shp
D:\lakemapping\sample750_pad\tif\sample_12903.tif
D:\lakemapping\2_dataset\sample747\output3\image_12903.tif
area_12926.shp
D:\lakemapping\sample750_pad\tif\sample_12926.tif
D:\lakemapping\2_dataset\sample747\output3\image_12926.tif
area_129

area_9553.shp
D:\lakemapping\sample750_pad\tif\sample_9553.tif
D:\lakemapping\2_dataset\sample747\output3\image_9553.tif
area_1974.shp
D:\lakemapping\sample750_pad\tif\sample_1974.tif
D:\lakemapping\2_dataset\sample747\output4\image_1974.tif
area_2155.shp
D:\lakemapping\sample750_pad\tif\sample_2155.tif
D:\lakemapping\2_dataset\sample747\output4\image_2155.tif
area_2618.shp
D:\lakemapping\sample750_pad\tif\sample_2618.tif
D:\lakemapping\2_dataset\sample747\output4\image_2618.tif
area_3560.shp
D:\lakemapping\sample750_pad\tif\sample_3560.tif
D:\lakemapping\2_dataset\sample747\output4\image_3560.tif
area_4685.shp
D:\lakemapping\sample750_pad\tif\sample_4685.tif
D:\lakemapping\2_dataset\sample747\output4\image_4685.tif
area_5006.shp
D:\lakemapping\sample750_pad\tif\sample_5006.tif
D:\lakemapping\2_dataset\sample747\output4\image_5006.tif
area_5026.shp
D:\lakemapping\sample750_pad\tif\sample_5026.tif
D:\lakemapping\2_dataset\sample747\output4\image_5026.tif
area_5711.shp
D:\lakemapping\sam

## 提取pad92个像素的shapefile
读取image影像，保存 pad width 为92个像素的shapefile

In [36]:
for i in range(0,type_num):
    writePath=os.path.join(output_dir,r'output{}'.format(i))
    shpWritePath=os.path.join(output_dir,r'shp\output{}'.format(i))
    
    all_files = []
    for root, dirs, files in os.walk(writePath):
        for file in files:
            if file.endswith(image_file_type)and file.startswith(image_prefix):
                 all_files.append((os.path.join(root, file), file))
    # print(all_files)

    for fullPath, filename in all_files:
        print(filename)
        shp_fn=filename.replace(image_file_type,area_file_type).replace(image_prefix,pad_area_prefix)
        print(shp_fn)
        dataset = gdal.Open(fullPath)
        geocd=dataset.GetGeoTransform()
        outfilename =os.path.join(shpWritePath,shp_fn)
        print(outfilename)
        oDriver = ogr.GetDriverByName('ESRI Shapefile')
        oDS = oDriver.CreateDataSource(outfilename)
        srs = osr.SpatialReference(wkt=dataset.GetProjection())
        oLayer = oDS.CreateLayer("polygon", srs, ogr.wkbPolygon)
        oDefn = oLayer.GetLayerDefn()
        row = dataset.RasterXSize+92
        line = dataset.RasterYSize+92
        geoxmin = geocd[0]-92*geocd[1]-92*geocd[2]
        geoymin = geocd[3]-92*geocd[4]-92*geocd[5]
        geoxmax = geocd[0] + (row) * geocd[1] + (line) * geocd[2]
        geoymax = geocd[3] + (row) * geocd[4] + (line) * geocd[5]
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(geoxmin, geoymin)
        ring.AddPoint(geoxmax, geoymin)
        ring.AddPoint(geoxmax, geoymax)
        ring.AddPoint(geoxmin, geoymax)
        ring.CloseRings()
        poly = ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)
        outfeat = ogr.Feature(oDefn)
        outfeat.SetGeometry(poly)
        oLayer.CreateFeature(outfeat)
        outfeat = None

image_10031.tif
pad_area_10031.shp
D:\lakemapping\2_dataset\sample747\shp\output0\pad_area_10031.shp


0

0

0

image_10150.tif
pad_area_10150.shp
D:\lakemapping\2_dataset\sample747\shp\output0\pad_area_10150.shp


...

0

image_8904.tif
pad_area_8904.shp
D:\lakemapping\2_dataset\sample747\shp\output5\pad_area_8904.shp


0

0

0

## 裁剪sampe影像，获得pad image影像

In [16]:
for i in range(0,type_num):
    all_files = []
    for root, dirs, files in os.walk(os.path.join(output_dir,'shp\output{}'.format(i))):
        for file in files:
            if file.endswith(area_file_type)and file.startswith(pad_area_prefix):
                 all_files.append((os.path.join(root, file), file))
#     print(all_files)                    
    for fullPath, filename in all_files:
        print(filename)
        raster_fn=os.path.join(raw_img_dir,filename.replace(pad_area_prefix,raw_image_prefix).replace(area_file_type,image_file_type))
        print(raster_fn)
        input_raster = gdal.Open(raster_fn)
        in_shp_path=fullPath
        out_raster_path=fullPath.replace(pad_area_prefix,pad_image_prefix).replace(area_file_type,image_file_type).replace(r'\shp','')
        print(out_raster_path)
            # 利用gdal.Warp进行裁剪
            # https://gdal.org/api/python/osgeo.gdal.html#osgeo.gdal.Warp
        result = gdal.Warp(
                out_raster_path,
                input_raster,
                format = 'GTiff',
                cutlineDSName = in_shp_path, # 用于裁剪的矢量
                cropToCutline = True, # 是否使用cutlineDSName的extent作为输出的界线
#                 dstNodata = -999 # 输出数据的nodata值
                )
        result.FlushCache()
        del result

pad_area_10031.shp
pad_area_10150.shp
pad_area_10174.shp
pad_area_10332.shp
pad_area_10358.shp
pad_area_10383.shp
pad_area_10408.shp
pad_area_10689.shp
pad_area_10713.shp
pad_area_10856.shp
pad_area_10892.shp
pad_area_10928.shp
pad_area_11020_3.shp
pad_area_11081.shp
pad_area_11289.shp
pad_area_11306.shp
pad_area_11324.shp
pad_area_11383.shp
pad_area_11428.shp
pad_area_11490.shp
pad_area_11601.shp
pad_area_11664.shp
pad_area_11740.shp
pad_area_11773.shp
pad_area_11904.shp
pad_area_12000.shp
pad_area_12234.shp
pad_area_12262.shp
pad_area_12277.shp
pad_area_12312.shp
pad_area_12344.shp
pad_area_12351.shp
pad_area_12389.shp
pad_area_12395.shp
pad_area_12443.shp
pad_area_12455.shp
pad_area_12596.shp
pad_area_1260.shp
pad_area_12602.shp
pad_area_12637.shp
pad_area_12666.shp
pad_area_12688.shp
pad_area_12821.shp
pad_area_12843.shp
pad_area_13177.shp
pad_area_13224.shp
pad_area_13246.shp
pad_area_13251.shp
pad_area_13307.shp
pad_area_13362.shp
pad_area_13407.shp
pad_area_13411_2.shp
pad_area_

## 生成annotation

In [9]:
#Read the training area 、 training polygons
trainingArea = gps.read_file(r'D:\lakemapping\2_dataset\sampleAnnotations\finalSample\area_type2345.shp')
trainingPolygon = gps.read_file(r'D:\lakemapping\2_dataset\sampleAnnotations\finalSample\polygon_type2345.shp')

print(trainingPolygon.shape,trainingArea.shape)# area:id, geomerry;   polygon:id, geometry 
trainingPolygon
trainingArea
# print(f'Read a total of {trainingPolygon.shape[0]} object polygons and {trainingArea.shape[0]} training areas.')
# print(f'Polygons will be assigned to training areas in the next steps.')

(216026, 2) (247, 6)


,CLASS_NAME,geometry
0,Lake,"POLYGON Z ((-66.54333 -2.42330 0.00000, -66.54..."
1,LakeLikeRiver,"POLYGON Z ((-66.49653 -2.42321 0.00000, -66.49..."
2,LakeLikeRiver,"POLYGON Z ((-66.49959 -2.42312 0.00000, -66.49..."
3,Lake,"POLYGON Z ((-66.43051 -2.42132 0.00000, -66.43..."
4,Lake,"POLYGON Z ((-66.55762 -2.43021 0.00000, -66.55..."
...,...,...
216021,Reservoir,"POLYGON Z ((-78.25458 21.51941 0.00000, -78.25..."
216022,Reservoir,"POLYGON Z ((33.00752 24.34722 0.00000, 33.0076..."
216023,Reservoir,"POLYGON Z ((33.00752 24.34722 0.00000, 33.0074..."
216024,Reservoir,"POLYGON Z ((33.02126 24.33500 0.00000, 33.0212..."


,Shape_Leng,Shape_Area,id,type,file_name,geometry
0,0.628580,229.401691,347,2,347,"POLYGON Z ((-4.99056 7.63742 0.00000, -5.13394..."
1,0.802842,489.156087,349,2,349,"POLYGON Z ((-5.71309 9.60074 0.00000, -5.51509..."
2,1.054339,841.794593,933,2,933,"POLYGON Z ((100.99123 4.57989 0.00000, 101.225..."
3,0.000000,209.589763,2080,2,2080,"POLYGON Z ((106.84582 -6.12265 0.00000, 107.00..."
4,0.000000,202.830576,2094,2,2094,"POLYGON Z ((108.81742 -7.00084 0.00000, 108.81..."
...,...,...,...,...,...,...
242,0.000000,521.261347,17683,5,17683,"POLYGON Z ((116.21661 -29.38319 0.00000, 116.4..."
243,0.000000,753.615692,17753,5,17753,"POLYGON Z ((123.22449 -29.09343 0.00000, 123.4..."
244,0.000000,31.153862,17883,5,17883,"POLYGON Z ((136.76231 -27.69697 0.00000, 136.8..."
245,0.000000,376.203141,18710,5,18710,"POLYGON Z ((-68.65232 -40.45139 0.00000, -68.6..."


In [10]:
# Check if the training areas and the training polygons have the same crs     
if trainingArea.crs  != trainingPolygon.crs:
    print('Training area CRS does not match training_polygon CRS')
    targetCRS = trainingPolygon.crs #Areas are less in number so conversion should be faster
    trainingArea = trainingArea.to_crs(targetCRS)
print(trainingPolygon.crs)
print(trainingArea.crs)
assert trainingPolygon.crs == trainingArea.crs

epsg:4326
epsg:4326


In [11]:
# As input we received two shapefile, first one contains the training areas/rectangles and other contains the polygon of lakes/objects in those training areas
# The first task is to determine the parent training area for each polygon.

def dividePolygonsInTrainingAreas(trainingPolygon, trainingArea):
    '''Assign annotated ploygons in to the training areas.'''
    # For efficiency, assigned polygons are removed from the list, we make a copy here. 
    cpTrainingPolygon = trainingPolygon.copy()
    splitPolygons = {}
    for i in tqdm(trainingArea.index):
        spTemp = [] 
        allocated = []
        print("area's index:",i)
        for j in cpTrainingPolygon.index:
            if cpTrainingPolygon.loc[j]['geometry'].within(trainingArea.loc[i]['geometry']):
                spTemp.append(cpTrainingPolygon.loc[j])
                allocated.append(j)      
        splitPolygons[i] = {'polygons':spTemp,'bounds':list(trainingArea.bounds.loc[i]),'file_name':trainingArea.loc[i]['file_name'] ,'type':trainingArea.loc[i]['type']}
        cpTrainingPolygon = cpTrainingPolygon.drop(allocated)#assigned polygons are removed from the list
    return splitPolygons

# areasWithPolygons contains the object polygons for each area!
areasWithPolygons = dividePolygonsInTrainingAreas(trainingPolygon, trainingArea)
print(f'Assigned training polygons in {len(areasWithPolygons)} training areas')

  0%|          | 0/247 [00:00<?, ?it/s]

area's index: 0
area's index: 1
area's index: 2
area's index: 3
area's index: 4
area's index: 5
area's index: 6
area's index: 7
area's index: 8
area's index: 9
area's index: 10
area's index: 11
area's index: 12
area's index: 13
area's index: 14
area's index: 15
area's index: 16
area's index: 17
area's index: 18
area's index: 19
area's index: 20
area's index: 21
area's index: 22
area's index: 23
area's index: 24
area's index: 25
area's index: 26
area's index: 27
area's index: 28
area's index: 29
area's index: 30
area's index: 31
area's index: 32
area's index: 33
area's index: 34
area's index: 35
area's index: 36
area's index: 37
area's index: 38
area's index: 39
area's index: 40
area's index: 41
area's index: 42
area's index: 43
area's index: 44
area's index: 45
area's index: 46
area's index: 47
area's index: 48
area's index: 49
area's index: 50
area's index: 51
area's index: 52
area's index: 53
area's index: 54
area's index: 55
area's index: 56
area's index: 57
area's index: 58
area's 

In [11]:
# print(areasWithPolygons) 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
def extractAreasThatOverlapWithTrainingData(areaInfo):
    """Iterates over raw NDWI images and using findOverlap() extract areas that overlap with training data. 
    The overlapping areas in raw images are written in a separate file, and annotation file are created from polygons in the overlapping areas.
    """   
    polygonsInAreaDf = gps.GeoDataFrame(areaInfo['polygons'])
    file_name=str(areaInfo['file_name'])
    bboxArea = box(*areaInfo['bounds'])
    area_type=areaInfo['type']
    writePath=os.path.join(output_dir,'output{}'.format(area_type))
    if not os.path.exists(writePath):
        os.makedirs(writePath)
    #draw image: set pad width of the sample images
    raw_img = rasterio.open(os.path.join(writePath,image_prefix+file_name+image_file_type))
    profile = raw_img.profile  
    profile['dtype'] = rasterio.uint8
    profile['nodata']= None
    profile['count'] = 1
    polygons = []
    for i in polygonsInAreaDf.index:
        gm = polygonsInAreaDf.loc[i]['geometry']
        polygons.append(gm)
                
    with rasterio.open(os.path.join(writePath,config.extracted_annotation_filename+'_{}.png'.format(file_name)), 'w+', **profile) as out:
        out_arr = out.read(1)
        burned = features.rasterize(polygons, fill=0, default_value=1,all_touched=False,out=out_arr, transform=out.transform)
        out.write_band(1, burned)

In [13]:
for value in areasWithPolygons.values():
    extractAreasThatOverlapWithTrainingData(value)

In [62]:
area_prefix='ndwi_'#'area_img_'
area_file_type='.shp'
sample_prefix='sample_'
sample_file_type='.tif'
output_prefix='image_'
sample_img_dir=r'D:\lakemapping\sample750_pad\tif'
all_files = []
for root, dirs, files in os.walk(os.path.join(output_dir,r'output5/shp')):
    for file in files:
        if file.endswith(area_file_type)and file.startswith(area_prefix):
             all_files.append((os.path.join(root, file), file))
print(all_files)

for fullPath, filename in all_files:
    print(filename)
    raster_fn=os.path.join(sample_img_dir,filename.replace(area_prefix,sample_prefix).replace(area_file_type,sample_file_type).replace('_pad92',''))
    print(raster_fn)
    input_raster = gdal.Open(raster_fn)
    in_shp_path=fullPath
    out_raster_path=fullPath.replace(area_prefix,output_prefix).replace(area_file_type,sample_file_type).replace('/shp','')
    print(out_raster_path)
        # 利用gdal.Warp进行裁剪
        # https://gdal.org/api/python/osgeo.gdal.html#osgeo.gdal.Warp
    result = gdal.Warp(
            out_raster_path,
            input_raster,
            format = 'GTiff',
            cutlineDSName = in_shp_path, # 用于裁剪的矢量
            cropToCutline = True, # 是否使用cutlineDSName的extent作为输出的界线
            dstNodata = -999 # 输出数据的nodata值
            )
    result.FlushCache()
    del result

[('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_10792_pad92.shp', 'ndwi_10792_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_10953_pad92.shp', 'ndwi_10953_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_16118_pad92.shp', 'ndwi_16118_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_17140_pad92.shp', 'ndwi_17140_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_17434_pad92.shp', 'ndwi_17434_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_17481_pad92.shp', 'ndwi_17481_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_17683_pad92.shp', 'ndwi_17683_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_17753_pad92.shp', 'ndwi_17753_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_17883_pad92.shp', 'ndwi_17883_pad92.shp'), ('D:\\lakemapping\\2_dataset\\test\\output5/shp\\ndwi_18710_pad92.shp', 'ndwi_18710_pad92.shp'), ('D:\\lakemapping\\2_dataset\

AttributeError: 'NoneType' object has no attribute 'FlushCache'

In [ ]:
# Display extracted image 
sampleImage = '_55.png'
# path_to_write=os.path.join(training_base_dir,'output\output4')
path_to_write=os.path.join(training_base_dir,'output\sample750\output0' )
fn = os.path.join(path_to_write, extracted_NDWI_filename + sampleImage)
NDWI_img = Image.open(fn)
read_NDWI_img = np.array(NDWI_img)

# redBands = os.path.join(path_to_write, 'red'+ sampleImage)
# red_img = Image.open(redBands)
# read_red_img = np.array(red_img)

greenBands = os.path.join(path_to_write, 'green'+ sampleImage)
green_img = Image.open(greenBands)
read_green_img = np.array(green_img)

# blueBands = os.path.join(path_to_write, 'blue'+ sampleImage)
# blue_img = Image.open(blueBands)
# read_blue_img = np.array(blue_img)

swirBands = os.path.join(path_to_write, 'swir'+ sampleImage)
swir_img = Image.open(swirBands)
read_swir_img = np.array(swir_img)
# print(read_NDWI_img.shape)
# print(read_Bands_img.shape)
annotation_im = Image.open(fn.replace(extracted_ndwi_filename ,config.extracted_annotation_filename))
read_annotation = np.array(annotation_im)
# print(read_annotation.shape)
# print(read_annotation)

all_images = np.array([read_NDWI_img,read_green_img,read_swir_img, read_annotation])#,read_red_img,read_blue_img
# print(all_images.shape[1])
display_images(np.expand_dims(np.transpose(all_images, axes=(1,2,0)), axis=0),['ndwi','green','swir','annotation'])